# Домашнее задание 3
## Сравнение интересов аудитории телеканалов НТВ и Дождь с помощью тематического моделирования LDA

Задача:
Сравнить интересы аудитории телеканолов НТВ и Дождь с помощью методов тематического моделирования
1. Получить данные по аудитории из социальной сети ВК
2. Зарегистрировать приложение, получить app_id, access_token
3. Скачать данные по пользователям в каждой из групп (id групп ВК даны ниже, tvrain_id, ntv_id)
4. Взять небольшую выборку из каждой совокупности телезрителей(около 1000-2000 человек, т.к. 300k-400k слишком много), с которыми работать дальше
5. Обучить LDA модель на их подписках
6. По группам, на которые подписаны эти люди, полуичть ключевые слова групп, на которые они подписаны
7. Получить распределение интересов людей для каждой совокупности, сравнить на графике

In [2]:
import seaborn as sb
import sys  
import pandas as pd
import requests
import numpy as np
import datetime
import time
import pickle
import matplotlib.pyplot as plt
import time
%matplotlib inline

from bs4 import BeautifulSoup as bs
import urllib

In [3]:
def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

#### Для использования VK API необходимо создать приложение в VK

1. Создать приложение по адресу https://vk.com/apps?act=manage (кнопка "создать приложение")
2. При создании указать название, описание (можно любые), категория  - прочее. Тип - standalone приложение
3. В настройках получить **app_id**. App_id потребуется для получения access token
4. Авторизовать пользователя (получить access token) можно по адресу: https://vk.com/dev/first_guide, в правилах нас интересует пункт 3 **Авторизация пользователя**
5. После того, как ознакомитесь с авторизацией пользователя, скопируйте в адресную строку такой запрос https://oauth.vk.com/authorize?client_id=5490057&display=page&redirect_uri=https://oauth.vk.com/blank.html&scope=friends&response_type=token&v=5.52, где число **5490057** замените на число, которое получите для вашего **app_id**
6. Нажмите Enter. Откроется окно с запросом прав. В нем отображаются название приложения, иконки прав доступа, и Ваши имя с фамилией. Нажмите «Разрешить». Вы попадете на новую страницу с предупреждением о том, что токен нельзя копировать и передавать третьим лицам. В адресной строке будет URL https://oauth.vk.com/blank.html, а после # Вы увидите дополнительные параметры — access_token, expires_in и user_id. Токен может выглядеть, например, так: 51eff86578a3bbbcb5c7043a122a69fd04dca057ac821dd7afd7c2d8e35b60172d45a26599c08034cc40a
7. Токен — это Ваш ключ доступа. При выполнении определенных условий человек, получивший Ваш токен, может нанести существенный ущерб Вашим данным и данным других людей. Поэтому очень важно не передавать свой токен третьим лицам
8. Поле expires_in содержит время жизни токена в секундах. 86400 секунд — это ровно сутки. Через сутки полученный токен перестанет действовать, для продолжения работы нужно будет получить новый по такому же алгоритму

In [4]:
# use your own app id and respective tokens

# скопируйте сюда ваши app_id и access_token, полученные по методу, описанному выше
app_id = '6978340'
access_token = '5b84d4c21c66483a85d532ff6028842fd21e9978a02127af537e4b5d97742cfd3de992e993681e5d5bee8'

# id групп ВК Дождя и НТВ
tvrain_id = 17568841
ntv_id = 28658784

In [5]:
# проверка работы API и авторизации пользователя. Если возникает ошибка, то, возможно, access token необходимо обновить
check_id = 2678564 #мой айди

# api call and test
def vk_get_response(method, parameters, token):
    url = 'https://api.vk.com/method/' + method + '?' + parameters + '&access_token=' + token
#     print url
    return(requests.get(url).json())

answer = vk_get_response(
    'users.get', 'user_ids={0}&v=4.9&lang=ru'.format(check_id), access_token
)['response']
print(answer[0]['first_name'], answer[0]['last_name'])

Александр Косолапов


### Получение подписчиков телеканалов НТВ и Дождь в VK

Получим объекты, которые содержат всю информацию о подпиичиках соответствующих групп (указанных в domains) и сохраним их на диск. Получим в итоге два файла - **ntv_subs** и **tvrain_subs** в формате **.pkl** - питоновский формат хранения данных

In [6]:
domains = ['ntv', 'tvrain']

for group_domain in domains:
    offset = 0
    group_id = group_domain
    fields = """sex,bdate,city,country,home_town,lists,domain,has_mobile,
    contacts,connections,education,universities,followers_count,occupation,last_seen,relation"""
    first_sample = vk_get_response(
        'groups.getMembers', 'group_id={0}&offset={1}&fields={2}&v=4.9&lang=ru'.format(
            group_id, offset, fields
        ), token=access_token
    )
    community_count = first_sample['response']['count']
    community_members = []
    for i in range(community_count // 2000 + 1):
        offset = i * 2000
        try:
            answer = vk_get_response(
                'groups.getMembers', 'group_id={0}&offset={1}&fields={2}&v=4.9&lang=ru'.format(
                    group_id, offset, fields), token=access_token
            )
            print("Offset: ", offset)
        except:
            print("Offset: ", offset, " Error")
        community_members += answer['response']['users']
    save_obj(community_members, '{}_subs'.format(group_domain))

Offset:  0
Offset:  2000
Offset:  4000
Offset:  6000
Offset:  8000
Offset:  10000
Offset:  12000
Offset:  14000
Offset:  16000
Offset:  18000
Offset:  20000
Offset:  22000
Offset:  24000
Offset:  26000
Offset:  28000
Offset:  30000
Offset:  32000
Offset:  34000
Offset:  36000
Offset:  38000
Offset:  40000
Offset:  42000
Offset:  44000
Offset:  46000
Offset:  48000
Offset:  50000
Offset:  52000
Offset:  54000
Offset:  56000
Offset:  58000
Offset:  60000
Offset:  62000
Offset:  64000
Offset:  66000
Offset:  68000
Offset:  70000
Offset:  72000
Offset:  74000
Offset:  76000
Offset:  78000
Offset:  80000
Offset:  82000
Offset:  84000
Offset:  86000
Offset:  88000
Offset:  90000
Offset:  92000
Offset:  94000
Offset:  96000
Offset:  98000
Offset:  100000
Offset:  102000
Offset:  104000
Offset:  106000
Offset:  108000
Offset:  110000
Offset:  112000
Offset:  114000
Offset:  116000
Offset:  118000
Offset:  120000
Offset:  122000
Offset:  124000
Offset:  126000
Offset:  128000
Offset:  130000
Of

In [8]:
community_ntv = load_obj('ntv_subs')
community_tvrain = load_obj('tvrain_subs')

In [9]:
community_ntv_df = pd.DataFrame(community_ntv)
community_tvrain_df = pd.DataFrame(community_tvrain)

community_ntv_df = community_ntv_df[community_ntv_df['deactivated'] != 'banned']
community_tvrain_df = community_tvrain_df[community_tvrain_df['deactivated'] != 'banned']

In [10]:
community_tvrain_df.head()

,bdate,city,country,deactivated,domain,education_form,education_status,facebook,facebook_name,faculty,...,occupation,relation,relation_partner,sex,skype,twitter,uid,universities,university,university_name
0,NaN,2.0,1.0,NaN,abacabadabacabaeabacabadabacaba,NaN,NaN,NaN,NaN,NaN,...,"{'type': 'university', 'id': 1, 'name': 'СПбГУ'}",NaN,NaN,2,NaN,NaN,6,NaN,NaN,NaN
1,NaN,NaN,NaN,deleted,id12,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2,NaN,NaN,12,NaN,NaN,NaN
2,2.6,2.0,1.0,NaN,id34,NaN,NaN,NaN,NaN,NaN,...,"{'type': 'university', 'id': 34, 'name': 'РГПУ...",NaN,NaN,1,NaN,tplutalova,34,NaN,NaN,NaN
3,14.7.1984,2.0,1.0,NaN,lesokhin,NaN,NaN,NaN,NaN,NaN,...,"{'type': 'work', 'name': 'Professional Communi...",NaN,NaN,2,NaN,NaN,46,NaN,NaN,NaN
4,26.3.1987,2.0,1.0,NaN,es,NaN,NaN,NaN,NaN,NaN,...,"{'type': 'work', 'id': 22822305, 'name': 'Кома...",NaN,NaN,1,NaN,NaN,243,NaN,NaN,NaN


Сначала получим всех уникальных подписчиков НТВ и Дождя с помощью unique. Далее с помощью numpy.random необходимо выбрать небольшой sample (например, по 1000 из каждой группы) таких людей и объединить их вместе

In [11]:
community_ntv_df_1000 = community_ntv_df[community_ntv_df['education_form'].notnull()]
community_tvrain_df_1000 = community_tvrain_df[community_tvrain_df['education_form'].notnull()]

ntv_uids = community_ntv_df_1000.uid.unique().tolist()
tvrain_uids = community_tvrain_df_1000.uid.unique().tolist()

In [12]:
# получить общий список людей из двух выборок НТВ и Дождя, всего должно быть в итоге около 2000 человек
import random
uids = random.choices(ntv_uids, k=1000) + random.choices(tvrain_uids, k=1000)
len(uids)

2000

In [24]:
len(set(uids))

1864

In [13]:
# получить подписки этих пользователей
print_counter = 0
final_data = []

#for uid in uids[:1000]:
for uid in uids:
    try:
        user_subs = vk_get_response(
            'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
        )
        time.sleep(0.3)
        final_data.append(user_subs)
    except:
        print("Error")
    if print_counter % 250 == 0:
        print("{0} profiles done".format(print_counter))
    print_counter += 1

0 profiles done
250 profiles done
500 profiles done
750 profiles done
1000 profiles done
1250 profiles done
1500 profiles done
1750 profiles done


In [30]:
subs_list = []
groups_freq_dict = {}
top_n = 5

for record, uid in zip(final_data, uids):
    try:
        user_subs = record
        if not user_subs.get('response'):
            user_subs = vk_get_response(
                'users.getSubscriptions', 'user_id={0}&v=4.9&lang=ru'.format(int(uid)), access_token
            )
        subs_pd = pd.DataFrame(
            [
                {
                    'groups_count': user_subs['response']['groups'].get('count'),
                    'groups_list': user_subs['response']['groups'].get('items'),
                    'follows_count':user_subs['response']['users'].get('count'),
                    'follows_list': user_subs['response']['users'].get('items'),
                }
            ]
        )

        for group_id in user_subs['response']['groups'].get('items')[:top_n]:
            if groups_freq_dict.get(group_id):
                groups_freq_dict[group_id] += 1
            else:
                groups_freq_dict[group_id] = 1

        subs_pd['subs_count'] = subs_pd['groups_count'] + subs_pd['follows_count']
        subs_list.append(subs_pd)
    except:
#         print(user_subs)
        pass
    if len(subs_list) % 500 == 0:
        print("Processed {0} users".format(len(subs_list)))

Processed 500 users
Processed 1000 users
Processed 1500 users


Самые популярные группы

In [31]:
sorted([(key, val) for key, val in groups_freq_dict.items()], key=lambda x: x[1], reverse=True)[:5]

[(17568841, 228),
 (28658784, 108),
 (29534144, 47),
 (40316705, 41),
 (15755094, 38)]

Загрузка постов со стен групп

In [33]:
group_doc_dict = {}
counter = 0
groups_freq_dict_top5 = groups_freq_dict

for group_id, freq in groups_freq_dict_top5.items():
    counter += 1
    try:
        check = vk_get_response(
            'wall.get',
            'owner_id={0}&count=100&fields=post_type,marked_as_ads&&v=4.9&lang=ru'.format(int(group_id) * -1),
            access_token
        )
        check = check['response']
        group_doc = ''
        if check[0] != 0:
            for post in check[1:]:
                if post.get('marked_as_ads') != 1:
                    group_doc += post['text']
        group_doc_dict[group_id] = group_doc
    except:
        print("Response error. Group id {0}".format(group_id))
        print(check)
    if counter % 100 == 0:
        print("{0} groups extracted".format(counter))
    time.sleep(0.3)

100 groups extracted
200 groups extracted
300 groups extracted
400 groups extracted
500 groups extracted
Response error. Group id 407755
{'error': {'error_code': 15, 'error_msg': 'Access denied: this wall available only for community members', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-407755'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
600 groups extracted
700 groups extracted
800 groups extracted
900 groups extracted
1000 groups extracted
1100 groups extracted
Response error. Group id 25705735
{'error': {'error_code': 15, 'error_msg': 'Access denied: this wall available only for community members', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-25705735'}, {'key': 'count', 'value': '100'}, {'key': 

Response error. Group id 78773303
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-78773303'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 41233427
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-41233427'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 83221497
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'

Response error. Group id 82862848
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-82862848'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 114793577
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-114793577'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 45919778
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, 

Response error. Group id 56595379
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-56595379'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 54069088
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-54069088'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 7783284
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'k

Response error. Group id 70409513
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-70409513'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 23872332
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-23872332'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 133154650
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {

...

Response error. Group id 3183750
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-3183750'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 90315329
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-90315329'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 23148449
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'ke

Response error. Group id 35228686
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-35228686'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 24323502
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-24323502'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 155227388
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {

Response error. Group id 55326998
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-55326998'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 78347663
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-78347663'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 180181897
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {

Response error. Group id 26127512
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-26127512'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 27243668
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-27243668'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 122456004
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {

Response error. Group id 68557282
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-68557282'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 33714790
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'key': 'method', 'value': 'wall.get'}, {'key': 'owner_id', 'value': '-33714790'}, {'key': 'count', 'value': '100'}, {'key': 'fields', 'value': 'post_type,marked_as_ads'}, {'key': '', 'value': ''}, {'key': 'v', 'value': '4.9'}, {'key': 'lang', 'value': 'ru'}]}}
Response error. Group id 64553733
{'error': {'error_code': 6, 'error_msg': 'Too many requests per second', 'request_params': [{'key': 'oauth', 'value': '1'}, {'

In [34]:
# сохранить сырые данные по постам групп на диск
save_obj(group_doc_dict, 'group_doc_dict')

In [35]:
from bs4 import BeautifulSoup
import re
import string
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
from pymorphy2 import MorphAnalyzer

In [36]:
chrs_to_delete = string.punctuation + u'»' + u'«' + u'—' + u'“' + u'„' + u'•' + u'#'
translation_table = {ord(c): None for c in chrs_to_delete if c != u'-'}
units = MorphAnalyzer.DEFAULT_UNITS
morph = MorphAnalyzer(result_type=None, units=units)
PortSt = PorterStemmer()
stopw = set(
    [w for w in stopwords.words(['russian', 'english'])]
    + [u'это', u'году', u'года', u'также', u'етот',
       u'которые', u'который', u'которая', u'поэтому',
       u'весь', u'свой', u'мочь', u'eтот', u'например',
       u'какой-то', u'кто-то', u'самый', u'очень', u'несколько',
       u'источник', u'стать', u'время', u'пока', u'однако',
       u'около', u'немного', u'кроме', u'гораздо', u'каждый',
       u'первый', u'вполне', u'из-за', u'из-под',
       u'второй', u'нужно', u'нужный', u'просто', u'большой',
       u'хороший', u'хотеть', u'начать', u'должный', u'новый', u'день',
       u'метр', u'получить', u'далее', u'именно', u'апрель',
       u'сообщать', u'разный', u'говорить', u'делать',
       u'появиться', u'2016',
       u'2015', u'получить', u'иметь', u'составить', u'дать', u'читать',
       u'ничто', u'достаточно', u'использовать',
       u'принять', u'практически',
       u'находиться', u'месяц', u'достаточно', u'что-то', u'часто',
       u'хотеть', u'начаться', u'делать', u'событие', u'составлять',
       u'остаться', u'заявить', u'сделать', u'дело',
       u'примерно', u'попасть', u'хотя', u'лишь', u'первое',
       u'больший', u'решить', u'число', u'идти', u'давать', u'вопрос',
       u'сегодня', u'часть', u'высокий', u'главный', u'случай', u'место',
       u'конец', u'работать', u'работа', u'слово', u'важный', u'сказать']
)

In [37]:
url_start = 'http[s]?://'
url_end = (
    '(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
)
pattern = url_start + url_end

Обработка слов постов групп - трансформация в "хороший" вид. Нормализация и стэмминг, удаление стоп-слов

In [ ]:
group_clean_doc_dict = {}
counter = 0

for group_id, doc in group_doc_dict.items():
    soup = BeautifulSoup(doc, 'html.parser')
    body = ' '.join(
        [tag.string.replace('\\n', ' ').replace('\\r', ' ')
         for tag in soup.descendants if tag.string]
    )
    body = re.sub('\[.*?\]','', body)
    body = re.sub(pattern,'', body)
    if body != '':
        body_clean = body.translate(translation_table).lower().strip()
        words = word_tokenize(body_clean)
        tokens = []
        # stemming and text normalization
        for word in words:
            if re.match('^[a-z0-9-]+$', word) is not None:
                tokens.append(PortSt.stem(word))
            elif word.count('-') > 1:
                tokens.append(word)
            else:
                normal_forms = morph.normal_forms(word)
                tokens.append(normal_forms[0] if normal_forms else word)
        # remove stopwords and leave unique words only
        tokens = filter(
            lambda token: token not in stopw, sorted(set(tokens))
        )

        # remove all words with more than 3 chars
        tokens = filter(lambda token: len(token) > 3, tokens)
    else:
        tokens = []
    counter += 1
    if counter % 100 == 0:
        print("{0} docs processed".format(counter))
    group_clean_doc_dict[group_id] = tokens

group_clean_doc_dict = {key: list(val) for key, val in group_clean_doc_dict.items()}

In [15]:
group_clean_doc_dict = load_obj('group_doc_dict_clean')

In [ ]:
# сохранить обработанные данные на диск
save_obj(group_clean_doc_dict, 'group_doc_dict_clean')

### Обучение LDA модели

In [17]:
!pip install gensim

  Running setup.py bdist_wheel for gensim: started
  Running setup.py bdist_wheel for gensim: finished with status 'done'
  Stored in directory: C:\Users\Aleksandr\AppData\Local\pip\Cache\wheels\73\6b\89\bb14fd56b74774a39a771a12f525a6a14c2c2692d3084ad048
  Running setup.py bdist_wheel for smart-open: started
  Running setup.py bdist_wheel for smart-open: finished with status 'done'
  Stored in directory: C:\Users\Aleksandr\AppData\Local\pip\Cache\wheels\b8\cb\43\c0ba52baf2b0e371ec1d5b2d4685d6d24617b1391f3eeacda5
Successfully built gensim smart-open


In [18]:
from gensim.corpora import TextCorpus
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.ldamodel import LdaModel

class ListTextCorpus(TextCorpus):

    def get_texts(self):
        for doc in self.input:
            yield doc
                
mycorp = ListTextCorpus(input=group_clean_doc_dict.values())
justlda = LdaModel(
    corpus=mycorp, num_topics=20, passes=12
)

In [19]:
print('LdaModel performance')
for i in range(20):
    terms = justlda.get_topic_terms(i)
    print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))

LdaModel performance
0 чтoб ecть мeнить тoлькo тeбить кoгдa знaть дeнь этoт кaждый
1 псевдоготический canal canyon bavaria dusk usaphi zealand parknew usafieri utah
2 человек группа друг город ждать жизнь праздник новое победа знать
3 окрашиваниестрижка жизни🙌🏻maybe мурашекбрать места⁉премьератор нашли😏 душутор 😎тор тaчкy inoi любовьпремьерасчастие
4 oпиcaниe пoявилиcь πpocтo пocт мoлoд пpoфeccие ужacнaть выпoлнeние cмoг пpиятeль
5 победыкак булoчка ушeй похoжий нaзвать совятаsoveckyjsouzфилинобыкновенныйsoveckyjsouzджунглевыйворобьиныйсычsoveckyjsouzбородатаянеясытьsoveckyjsouzбелолицаясовкаsoveckyjsouzдобрый совяткиушастаясоваsoveckyjsouzфилининдийскийsoveckyjsouzсераянеясытьsoveckyjsouzкроличийсычsoveckyjsouzястребинаясоваsoveckyjsouzдобрый совяткиушастаясоваsoveckyjsouzфилининдийскийsoveckyjsouzсипухаsoveckyjsouzсераянеясытьsoveckyjsouz совятаsoveckyjsouzфилинсибирскийsoveckyjsouzбелолицаясовкаsoveckyjsouzбородатаянеясытьsoveckyjsouzворобьиныйсычикsoveckyjsouzдобрый птенцысовsovec

In [20]:
dummy_dict = {key: 0 for key in range(20)}

group_topics_dict_20 = {
    group_id: dict(list(dummy_dict.items()) + justlda.get_document_topics(mycorp.dictionary.doc2bow(text)))
    for group_id, text in group_clean_doc_dict.items()
}
check_pd_20 = pd.DataFrame.from_dict(group_topics_dict_20, orient='index')
check_pd_20.head(10)
print("Group distribution by the most relevant topic")
pd.Series.round(check_pd_20.idxmax(axis=1).value_counts() * 1. / len(check_pd_20), 2)

Group distribution by the most relevant topic


2     0.67
15    0.20
12    0.04
0     0.04
13    0.03
19    0.01
3     0.00
14    0.00
6     0.00
4     0.00
1     0.00
8     0.00
11    0.00
5     0.00
17    0.00
16    0.00
9     0.00
18    0.00
7     0.00
10    0.00
dtype: float64

In [21]:
# dump lda model to disk
justlda.save('ldamodel_20_topics')

In [22]:
print("The most typical groups for every topic")
for i in range(20):
    terms = justlda.get_topic_terms(i)
    print(i, ' '.join(map(lambda x: mycorp.dictionary.get(x[0]), terms)))
    typical_groups = check_pd_20[i].sort_values(ascending=False).index[:10]
    for g in typical_groups:
        group_info = vk_get_response(
            'groups.getById', 'group_ids={0}&v=4.9&lang=ru'.format(g), access_token
        )
        print(group_info['response'][0]['name'] + ' ' + 'http://vk.com/club' + str(g))
        time.sleep(0.3)
    print()

The most typical groups for every topic
0 чтoб ecть мeнить тoлькo тeбить кoгдa знaть дeнь этoт кaждый
Криминал | Crime http://vk.com/club129012690
Самое интересное | Бот http://vk.com/club11787
Интересные факты и Приколы http://vk.com/club33376541
Гифки http://vk.com/club44054463
Интересные Факты http://vk.com/club346191
Lady Bla http://vk.com/club113670591
Новинки Музыки 2019 http://vk.com/club54500021
Men's Time http://vk.com/club59878292
В Гараже -  Паблик для автовладельцев http://vk.com/club157226535
Basota. http://vk.com/club8251727

1 псевдоготический canal canyon bavaria dusk usaphi zealand parknew usafieri utah
Outstanding | Путешествия, Фото http://vk.com/club169184526
AMEN FASHION http://vk.com/club24483645
СВАТЫ http://vk.com/club32891993
DIE http://vk.com/club39684469
Краткие факты http://vk.com/club75149440
ЯНИТАКАЯ http://vk.com/club167140804
Алло -такси п.Колышлей http://vk.com/club168044454
Как Челентано http://vk.com/club159946402
СЕЛЯНСТВО http://vk.com/club162180441

Копилка кулинарных рецептов | GIF рецепты http://vk.com/club106501715
Худей вкусно http://vk.com/club35113021
Рецепты: вкусно и просто🍴 http://vk.com/club46117626
Рецепты для мультиварки http://vk.com/club43706510

14 299o кpутуй alрine kmm-bt304 сde-193 рiоneer mvн-х58оbт мaгнитол arrow сумеречныеохотник
Знакомства TOPFACE http://vk.com/club25678227
МК18+ http://vk.com/club24532152
Söylə.. http://vk.com/club24577388
OLDSCHOOL GARAGE http://vk.com/club35673445
Auto Drive | Авто и техника http://vk.com/club159415165
ШВТ http://vk.com/club152219848
Академия Джентльменов http://vk.com/club86990325
Female Beauty http://vk.com/club181934527
JASKIER |Сериалы и Кино| ЯСКЪЕР http://vk.com/club47254649
ЯЖЕВОДИТЕЛЬ http://vk.com/club151016287

15 любить твой любовь глаз жизнь жить человек понять сердце знать
Лучшие стихи великих поэтов | Литература http://vk.com/club38683579
Tears of Memory http://vk.com/club42461968
Литературный кайф http://vk.com/club90983046
Путь мудреца http://vk.com/club157